In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("dados/raw_data_with_labels.csv", index_col=0)
df = df[df['y'].notnull()]
df.shape

(772, 20)

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [7]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']
df_limpo['title'] 

link
watch?v=eUslDxZfjGc    Hands-on Training in Artificial Intelligence w...
watch?v=0f60PduPRYE    How to LEARN DATA SCIENCE & Machine Learning f...
watch?v=nFzgPVwr5vo                       Data Science Indonesia Profile
watch?v=tgS3NXMfmI0    Hands-On Training in Artificial Intelligence w...
watch?v=mO47WN4PT_Y     Hands-On Training in AI with DataScience | Day-5
                                             ...                        
watch?v=wC4YV8t40zU    Talks # 5: Parul Pandey: Data Science, Diversi...
watch?v=62HBi0B3se0    Ep.04 "My Journey Into Data Science" with Abhi...
watch?v=R8gLOuGYW8w          Predict Future Sales | Kaggle | Code review
watch?v=VeM1T7UaYTk    Maximilian Jeblick | Physics, Math and Data Sc...
watch?v=PSvEh8ukf-c    Uploading dataset from Kaggle directly to Goog...
Name: title, Length: 772, dtype: object

## 1. Limpeza da data

In [8]:
df_limpo['date'] = pd.to_datetime(df['upload_date'], format='%Y-%m-%d')
df_limpo['date']

link
watch?v=eUslDxZfjGc   2020-11-17
watch?v=0f60PduPRYE   2020-11-15
watch?v=nFzgPVwr5vo   2020-11-14
watch?v=tgS3NXMfmI0   2020-11-12
watch?v=mO47WN4PT_Y   2020-11-10
                         ...    
watch?v=wC4YV8t40zU   2020-06-19
watch?v=62HBi0B3se0   2020-06-18
watch?v=R8gLOuGYW8w   2020-06-14
watch?v=VeM1T7UaYTk   2020-06-11
watch?v=PSvEh8ukf-c   2020-06-16
Name: date, Length: 772, dtype: datetime64[ns]

## 2. Limpeza de Views

In [9]:
df_limpo['views'] = df['view_count'].map(lambda x: 0 if x<0 else int(x))
df_limpo['views']

link
watch?v=eUslDxZfjGc    379
watch?v=0f60PduPRYE    124
watch?v=nFzgPVwr5vo     13
watch?v=tgS3NXMfmI0    655
watch?v=mO47WN4PT_Y      1
                      ... 
watch?v=wC4YV8t40zU      3
watch?v=62HBi0B3se0      2
watch?v=R8gLOuGYW8w      1
watch?v=VeM1T7UaYTk    201
watch?v=PSvEh8ukf-c    140
Name: views, Length: 772, dtype: int64

## 3. Features

In [10]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [11]:
data_extract = datetime.datetime.strptime('2020-11-21', "%Y-%m-%d")

In [12]:
features['tempo_desde_pub'] = (data_extract - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [14]:
features.head()

,views,views_por_dia
link,,
watch?v=eUslDxZfjGc,379,94.750000
watch?v=0f60PduPRYE,124,20.666667
watch?v=nFzgPVwr5vo,13,1.857143
watch?v=tgS3NXMfmI0,655,72.777778
watch?v=mO47WN4PT_Y,1,0.090909


In [15]:
data_corte = "2020-10-01"
mask_train = df_limpo['date'] < data_corte
mask_val = df_limpo['date'] >= data_corte

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((385, 2), (387, 2), (385,), (387,))

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=1)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [30]:
title_bow_train.shape

(385, 1133)

In [31]:
1 - 2595/(385*385)

0.9824928318434812

In [32]:
title_bow_train

<385x1133 sparse matrix of type '<class 'numpy.float64'>'
	with 3382 stored elements in Compressed Sparse Row format>

In [21]:
hstack - [1 2]     [3 4]   -> [1 2 3 4] - 1x4

vstack - [1 2]     [3 4]   -> [1 2]
                              [3 4] - 2x2

SyntaxError: invalid syntax (<ipython-input-21-ab0626dc6b61>, line 1)

In [33]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [34]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((385, 1135), (387, 1135))

In [35]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [36]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [37]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [38]:
average_precision_score(yval, p)

0.5235463345585548

In [39]:
roc_auc_score(yval, p)

0.665264593872563

In [ ]:
ap 0.5235463345585548, auc 0.665264593872563 - mindf=1
ap 0.5332331171914984, auc 0.6808673904393794 - mindf=2

# 5 Active learning

138 exemplos que o modelo tenha dificuldade
62 exemplos aleatoriamente

In [33]:
df_unlabeled = pd.read_csv("raw_data_with_labels.csv")
df_unlabeled = df_unlabeled[df_unlabeled['y'].isnull()].dropna(how='all')
df_unlabeled.shape

(1217, 21)

In [34]:
df_unlabeled.head(1)

,link,webpage_url,title,y,query,view_count,like_count,dislike_count,average_rating,subscriber_count,description,categories,tags,upload_date,tempo_desde_pub,channel_url,thumbnail,width,height,resolution,fps
156,watch?v=vHaIk07i3zw,https://www.youtube.com/watch?v=vHaIk07i3zw,Improving Global Health Forecasting: Data Scie...,NaN,data+science,349,5.0,NaN,5.0,6640.0,Providing patients/end customers with high-qua...,['Nonprofits & Activism'],"['CGD', 'Center for Global Development', 'rese...",2020-11-12,5,http://www.youtube.com/channel/UCWC4mmS_a-0K6J...,https://i.ytimg.com/vi/vHaIk07i3zw/maxresdefau...,1280,720,NaN,60


In [38]:
df_limpo_u = pd.DataFrame(index=df_unlabeled.index)
df_limpo_u['title'] = df_unlabeled['title']

In [39]:
df_limpo_u['date'] = pd.to_datetime(df_unlabeled['upload_date'], format='%Y-%m-%d')

In [40]:
df_limpo_u.head()

,title,date
156,Improving Global Health Forecasting: Data Scie...,2020-11-12
157,Data Science in Africa: Live Q&A with Chris Or...,2020-11-12
158,Hands-On Training in Artificial Intelligence w...,2020-11-12
159,Tableau Data Science Tutorial | Tableau for DS...,2020-11-11
160,PYTHON ou R para Data Science?,2020-11-11


In [41]:
df_limpo_u['views'] = df_unlabeled['view_count'].map(lambda x: 0 if x<0 else int(x))

In [42]:
features_u = pd.DataFrame(index=df_limpo_u.index)

In [43]:
features_u['tempo_desde_pub'] = (datetime.datetime.today() - df_limpo_u['date']) / np.timedelta64(1, 'D')
features_u['views'] = df_limpo_u['views']
features_u['views_por_dia'] = features_u['views'] / features_u['tempo_desde_pub']
features_u = features_u.drop(['tempo_desde_pub'], axis=1)

In [44]:
features_u.head()

,views,views_por_dia
156,349,50.294794
157,1,0.144111
158,655,94.392809
159,2,0.251918
160,1,0.125959


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_u = df_limpo_u['title']
title_bow_u = title_vec.transform(title_u)


In [46]:
title_bow_u

<1217x860 sparse matrix of type '<class 'numpy.float64'>'
	with 7014 stored elements in Compressed Sparse Row format>

In [47]:
Xu_wtitle = hstack([features_u, title_bow_u])

In [48]:
Xu_wtitle

<1217x862 sparse matrix of type '<class 'numpy.float64'>'
	with 9444 stored elements in COOrdinate format>

In [49]:
pu = mdl.predict_proba(Xu_wtitle)[:, 1]

In [50]:
df_unlabeled['p'] = pu

In [53]:
df_unlabeled.head(1)

,link,webpage_url,title,y,query,view_count,like_count,dislike_count,average_rating,subscriber_count,description,categories,tags,upload_date,tempo_desde_pub,channel_url,thumbnail,width,height,resolution,fps,p
156,watch?v=vHaIk07i3zw,https://www.youtube.com/watch?v=vHaIk07i3zw,Improving Global Health Forecasting: Data Scie...,NaN,data+science,349,5.0,NaN,5.0,6640.0,Providing patients/end customers with high-qua...,['Nonprofits & Activism'],"['CGD', 'Center for Global Development', 'rese...",2020-11-12,5,http://www.youtube.com/channel/UCWC4mmS_a-0K6J...,https://i.ytimg.com/vi/vHaIk07i3zw/maxresdefau...,1280,720,NaN,60,0.141


In [55]:
mask_u = (df_unlabeled['p'] >= 0.45) & (df_unlabeled['p'] <= 0.55)
mask_u.sum()

138

In [56]:
mask_u

156     False
157     False
158     False
159     False
160      True
        ...  
1984     True
1985    False
1986     True
1987    False
1988    False
Name: p, Length: 1217, dtype: bool

In [57]:
df_unlabeled[mask_u]#.sort_values("p")

,link,webpage_url,title,y,query,view_count,like_count,dislike_count,average_rating,subscriber_count,description,categories,tags,upload_date,tempo_desde_pub,channel_url,thumbnail,width,height,resolution,fps,p
160,watch?v=_m0UanRL6FE,https://www.youtube.com/watch?v=_m0UanRL6FE,PYTHON ou R para Data Science?,NaN,data+science,1,147.0,2.0,4.972973,5310.0,Qual a melhor linguagem para,['Science & Technology'],NaN,2020-11-11,6,http://www.youtube.com/channel/UCxaCu5ht7d3jvi...,https://i.ytimg.com/vi_webp/_m0UanRL6FE/maxres...,1920,1080,NaN,24,0.541
179,watch?v=1x9xGVRs-5E,https://www.youtube.com/watch?v=1x9xGVRs-5E,Hands-On Training In AI with Data Science,NaN,data+science,4,114.0,5.0,4.831932,14600.0,NaN,['Entertainment'],NaN,2020-11-04,13,http://www.youtube.com/channel/UCF2_gALht1C1Ns...,https://i.ytimg.com/vi/1x9xGVRs-5E/maxresdefau...,1280,720,NaN,30,0.511
191,watch?v=PYap8CU0VaA,https://www.youtube.com/watch?v=PYap8CU0VaA,Linux on the Desktop for the Data Science (wit...,NaN,data+science,1,52.0,NaN,5.000000,41700.0,Most,['Science & Technology'],"['linux', 'jeff heaton', 'data science', 'mach...",2020-10-27,21,http://www.youtube.com/channel/UCR1-GEpyOPzT2A...,https://i.ytimg.com/vi_webp/PYap8CU0VaA/maxres...,1920,1080,NaN,30,0.490
194,watch?v=e4i_EGAa0gU,https://www.youtube.com/watch?v=e4i_EGAa0gU,How to get a job after learning data science s...,NaN,data+science,5,391.0,6.0,4.939547,190000.0,Many times people complain that we have learnt...,['Education'],"['yt:cc=on', 'how to get a job as data scienti...",2020-10-25,23,http://www.youtube.com/channel/UCh9nVJoWXmFb7s...,https://i.ytimg.com/vi_webp/e4i_EGAa0gU/maxres...,3840,2160,NaN,30,0.496
207,watch?v=Tm0YyhAOOok,https://www.youtube.com/watch?v=Tm0YyhAOOok,free course with certificate | Microsoft free ...,NaN,data+science,63,2257.0,64.0,4.889703,47100.0,free course with certificate | Microsoft free ...,['Science & Technology'],"['free course', 'free certificate', 'free cour...",2020-10-18,30,http://www.youtube.com/channel/UCNr_dWKxRanxFU...,https://i.ytimg.com/vi_webp/Tm0YyhAOOok/maxres...,1280,720,NaN,30,0.529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980,watch?v=O1P4r0Iy55U,https://www.youtube.com/watch?v=O1P4r0Iy55U,Getting Started on Kaggle: A quick tour | Kaggle,NaN,kaggle,19,198.0,2.0,4.979899,68900.0,"In this video,",['Science & Technology'],"['kaggle', 'data science', 'Kaggel', 'coffee c...",2018-10-16,763,http://www.youtube.com/channel/UCSNeZleDn9c74y...,https://i.ytimg.com/vi_webp/O1P4r0Iy55U/maxres...,1920,1006,NaN,29,0.451
1981,watch?v=sLAthlX816c,https://www.youtube.com/watch?v=sLAthlX816c,Getting Started on Kaggle: Python coding in Ke...,NaN,kaggle,26,275.0,9.0,4.873239,68900.0,"In this video,",['Science & Technology'],"['kaggle', 'data science', 'coding', 'python',...",2018-10-16,763,http://www.youtube.com/channel/UCSNeZleDn9c74y...,https://i.ytimg.com/vi_webp/sLAthlX816c/maxres...,1280,672,NaN,29,0.537
1982,watch?v=0jQwAp7po00,https://www.youtube.com/watch?v=0jQwAp7po00,Getting Started on Kaggle: Uploading data to a...,NaN,kaggle,11,71.0,4.0,4.786667,68900.0,"In this video,",['Science & Technology'],"['kaggle', 'data science', 'python', 'data', '...",2018-10-16,763,http://www.youtube.com/channel/UCSNeZleDn9c74y...,https://i.ytimg.com/vi_webp/0jQwAp7po00/maxres...,1920,1006,NaN,29,0.472
1984,watch?v=fvF2H85ko9c,https://www.youtube.com/watch?v=fvF2H85ko9c,Getting Started on Kaggle: Writing code to ana...,NaN,kaggle,35,464.0,12.0,4.899159,68900.0,"In this video,",['Science & Technology'],"['kaggle', 'data science', 'data', 'python', '...",2018-10-16,763,http://www.youtube.com/channel/UCSNeZleDn9c74y...,https://i.ytimg.com/vi_webp/fvF2H85ko9c/maxres...,1280,674,NaN,29,0.521


In [58]:
dificeis = df_unlabeled[mask_u]

In [60]:
aleatorios = df_unlabeled[~mask_u].sample(62, random_state=0)

In [61]:
pd.concat([dificeis, aleatorios]).to_csv("active_label1.csv", index=None)

In [62]:
dificeis.head()

,link,webpage_url,title,y,query,view_count,like_count,dislike_count,average_rating,subscriber_count,description,categories,tags,upload_date,tempo_desde_pub,channel_url,thumbnail,width,height,resolution,fps,p
160,watch?v=_m0UanRL6FE,https://www.youtube.com/watch?v=_m0UanRL6FE,PYTHON ou R para Data Science?,NaN,data+science,1,147.0,2.0,4.972973,5310.0,Qual a melhor linguagem para,['Science & Technology'],NaN,2020-11-11,6,http://www.youtube.com/channel/UCxaCu5ht7d3jvi...,https://i.ytimg.com/vi_webp/_m0UanRL6FE/maxres...,1920,1080,NaN,24,0.541
179,watch?v=1x9xGVRs-5E,https://www.youtube.com/watch?v=1x9xGVRs-5E,Hands-On Training In AI with Data Science,NaN,data+science,4,114.0,5.0,4.831932,14600.0,NaN,['Entertainment'],NaN,2020-11-04,13,http://www.youtube.com/channel/UCF2_gALht1C1Ns...,https://i.ytimg.com/vi/1x9xGVRs-5E/maxresdefau...,1280,720,NaN,30,0.511
191,watch?v=PYap8CU0VaA,https://www.youtube.com/watch?v=PYap8CU0VaA,Linux on the Desktop for the Data Science (wit...,NaN,data+science,1,52.0,NaN,5.000000,41700.0,Most,['Science & Technology'],"['linux', 'jeff heaton', 'data science', 'mach...",2020-10-27,21,http://www.youtube.com/channel/UCR1-GEpyOPzT2A...,https://i.ytimg.com/vi_webp/PYap8CU0VaA/maxres...,1920,1080,NaN,30,0.490
194,watch?v=e4i_EGAa0gU,https://www.youtube.com/watch?v=e4i_EGAa0gU,How to get a job after learning data science s...,NaN,data+science,5,391.0,6.0,4.939547,190000.0,Many times people complain that we have learnt...,['Education'],"['yt:cc=on', 'how to get a job as data scienti...",2020-10-25,23,http://www.youtube.com/channel/UCh9nVJoWXmFb7s...,https://i.ytimg.com/vi_webp/e4i_EGAa0gU/maxres...,3840,2160,NaN,30,0.496
207,watch?v=Tm0YyhAOOok,https://www.youtube.com/watch?v=Tm0YyhAOOok,free course with certificate | Microsoft free ...,NaN,data+science,63,2257.0,64.0,4.889703,47100.0,free course with certificate | Microsoft free ...,['Science & Technology'],"['free course', 'free certificate', 'free cour...",2020-10-18,30,http://www.youtube.com/channel/UCNr_dWKxRanxFU...,https://i.ytimg.com/vi_webp/Tm0YyhAOOok/maxres...,1280,720,NaN,30,0.529


In [63]:
aleatorios.head()

,link,webpage_url,title,y,query,view_count,like_count,dislike_count,average_rating,subscriber_count,description,categories,tags,upload_date,tempo_desde_pub,channel_url,thumbnail,width,height,resolution,fps,p
1501,watch?v=OLzfr1oJXTE,https://www.youtube.com/watch?v=OLzfr1oJXTE,Machine Learning (Aprendizado de Máquina) - Co...,NaN,aprendizado+de+maquina,2,32.0,NaN,5.000000,2200.0,ENSINO -,['Education'],"['mineração de dados', 'aprendizado de máquina...",2014-05-07,2386,http://www.youtube.com/channel/UCOx5f4MPcU3KYe...,https://i.ytimg.com/vi/OLzfr1oJXTE/maxresdefau...,640,480,NaN,30,0.051
1685,watch?v=9iW8TXkzrto,https://www.youtube.com/watch?v=9iW8TXkzrto,What is Kaggle | How to use Kaggle in hindi,NaN,kaggle,1,31.0,2.0,4.757576,3330.0,in this video understand how to use,['Education'],"['What', 'Is', 'Kaggle', 'What is kaggle', 'Wh...",2020-06-06,164,http://www.youtube.com/channel/UCPIMmfCnHXcaR8...,https://i.ytimg.com/vi_webp/9iW8TXkzrto/maxres...,1278,720,NaN,24,0.242
1412,watch?v=oEdFzRkk0Hk,https://www.youtube.com/watch?v=oEdFzRkk0Hk,SETI UFLA - Palestra Aprendizado de Máquina Ap...,NaN,aprendizado+de+maquina,48,2.0,NaN,5.000000,268.0,Pesquisas relevantes na área,['Science & Technology'],NaN,2018-09-26,783,http://www.youtube.com/channel/UCHtvp-d8pujKyA...,https://i.ytimg.com/vi/oEdFzRkk0Hk/hqdefault.j...,1280,720,NaN,60,0.350
1062,watch?v=SfOoRsUj9kQ,https://www.youtube.com/watch?v=SfOoRsUj9kQ,Intelligence Artificielle VS Machine Learning ...,NaN,machine+learning,16,375.0,10.0,4.896104,7060.0,Aujourd'hui on parles des différences qui rési...,['Science & Technology'],"['IA', 'Inteligence Artificielle', 'Machine Le...",2018-10-10,769,http://www.youtube.com/channel/UChXy935RrhKDL3...,https://i.ytimg.com/vi_webp/SfOoRsUj9kQ/maxres...,1920,1080,NaN,60,0.441
1398,watch?v=lVQ1YYqQP8Q,https://www.youtube.com/watch?v=lVQ1YYqQP8Q,Minicurso Aprendizado de Máquina - SESCOMP 201...,NaN,aprendizado+de+maquina,37,NaN,NaN,NaN,859.0,Ciência,['Science & Technology'],"['Numpy', 'Ciência de dados', 'Pandas', 'KNN',...",2018-11-26,722,http://www.youtube.com/channel/UCnR_-6nHlN-RrK...,https://i.ytimg.com/vi_webp/lVQ1YYqQP8Q/maxres...,1280,720,NaN,30,0.193
